In [1]:
import os
import scipy.stats
import pandas as pd
import numpy as np
import math
from prettytable import PrettyTable
from operator import itemgetter
import matplotlib.pyplot as plt

## Computing number of samples as a function of epsilon and the other way around

In [2]:
def compute_n(c, delta, epsilon, size_of_market):
    return (c / epsilon) * (c / epsilon) * 0.5 * math.log((2 * size_of_market) / delta)

In [3]:
def compute_eps(c, delta, size_of_market, number_of_samples):
    return c * math.sqrt((math.log((2.0 * size_of_market) / delta)) / (2.0 * number_of_samples))

In [4]:
#candidate_eps = [0.125, 0.25, 0.5, 1.0]
candidate_eps = [10.0 / (2 ** i) for i in range(0, 4)]
for eps in candidate_eps:
    m_eps = compute_n(402, 0.1, eps, 4480)
    print(eps, [int(m_eps /4) , int(m_eps / 2), int(m_eps), int(2 * m_eps)])

10.0 [2303, 4606, 9213, 18427]
5.0 [9213, 18427, 36855, 73711]
2.5 [36855, 73711, 147423, 294846]
1.25 [147423, 294846, 589692, 1179384]


### Testing eps to n and n to eps functions

In [5]:
c = 2
delta = 0.1
epsilon = 1.0
size_of_market = 4480
n = compute_n(c, delta, epsilon, size_of_market)
assert epsilon == compute_eps(c, delta, size_of_market, n)
assert n == compute_n(c, delta, epsilon, size_of_market)

## Compute Statistics from LSVM worlds

In [6]:
def compare_ea_eap(loc, world_number, epsilon):
    # Read EAP data.
    bidders_final_values_df = pd.read_csv(f"{loc}bidders_final_values.csv")
    eap_samples = bidders_final_values_df['n'].sum()
    
    # Compute EA data.
    bidders_summary_df = pd.read_csv(f"{loc}bidders_summary.csv")
    params_csv = pd.read_csv(f"{loc}params.csv")
    c, delta, eps_eap = int(params_csv.iloc[3]['value']), sum(eval(params_csv.iloc[1]['value'])), float(params_csv.iloc[6]['value'])
    size_of_market = sum([2 ** len(eval(row[2])) for row in bidders_summary_df.itertuples()])
    ea_samples = compute_n(c, delta, epsilon, size_of_market) * size_of_market
    
    # Make sure that the epsilon given as parameters coincides with the epsilon actually needed by EA. 
    assert math.isclose(epsilon, compute_eps(c, delta, size_of_market, ea_samples / size_of_market))
    
    return [world_number, 
            c, 
            delta, 
            epsilon, 
            eps_eap, 
            size_of_market,
            ea_samples, 
            eap_samples,
            (eap_samples / ea_samples) * 100.0]

In [7]:
def construct_ea_eap_ptable(results, sort_key=7):
    # Sort by desired key
    results = sorted(results, key=itemgetter(sort_key))
    # Construct ptable
    results_ptable = PrettyTable()
    results_ptable.field_names = ['world', 'c', 'delta', 'epsilon', 'eap_epsilon', 'size_of_market', 'ea', 'eap', 'ratio']
    results_ptable.align["ea"] = "r"
    results_ptable.align["eap"] = "r"

    for r in results:
        results_ptable.add_row([r[0], 
                                r[1], 
                                r[2], 
                                f"{r[3] : .4f}", 
                                f"{r[4] : .4f}", 
                                f"{r[5] : ,}", 
                                f"{r[6] : ,.0f}", 
                                f"{r[7] : ,.0f}", 
                                f"{r[8] : .4f}"])
    return results_ptable

#Collect results from all experiments. 
# results_dir = '../experiments/LSVM/'
# results = []
# for e in [0, 1]:
#     for i in range(0, 11):
#         loc = f"{results_dir}{e}/worlds_results/world{i}/eps_0.125/"
#         results += [compare_ea_eap(loc, i)]
# print(construct_ea_eap_ptable(results))

## Table Data

In [8]:
def compute_conf_interval(sample, confidence_level = 0.95):
    degrees_freedom = len(sample) - 1
    sample_mean = np.mean(sample)
    sample_standard_error = scipy.stats.sem(sample)
    lb, ub = scipy.stats.t.interval(confidence_level, degrees_freedom, sample_mean, sample_standard_error)

    return sample_mean, lb, ub, (ub - lb)/2

confidence = 0.95
for eps in ['0.125', '0.25', '0.5', '1.0']:
    #data = [compare_ea_eap(f"../experiments/GSVM/{e}/worlds_results/world{i}/eps_{eps}/", i, float(eps)) for i in range(0, 30) for e in [0, 1]]
    data = [compare_ea_eap(f"../experiments/LSVM/{e}/worlds_results/world{i}/eps_{eps}/", i, float(eps)) for i in range(0, 11) for e in [0, 1]]
    eps_eap = [data[4] for data in data]
    num_samples_eap = [data[7] for data in data]
    size_of_market = [compute_n(c=int(data[1]),
                                delta=float(data[2]), 
                                epsilon=float(eps), 
                                size_of_market = int(data[5])) * int(data[5]) for data in data]
    
    # Compute confidence intervals. 
    mean_eap_eps, lb_eap_eps, ub_eap_eps, width_ci_eap_eps = compute_conf_interval(eps_eap, confidence)
    mean_eap, lb_eap, ub_eap, width_ci_eap = compute_conf_interval(num_samples_eap, confidence)
    mean_ea, lb_ea, ub_ea, width_ci_ea = compute_conf_interval(size_of_market, confidence)
    
    # Prints. 
    print(f"{eps} ", end='')
    print(f"${mean_ea / 1000 :,.0f} \pm {width_ci_ea / 1000 :,.2f}$ & ", end='')
    print(f"${mean_eap / 1000 :,.0f} \pm {width_ci_eap / 1000 :,.2f}$ &", end='')
    print(f"${mean_eap_eps :.4f} \pm {width_ci_eap_eps :.4f}$ ")


0.125 $524,487 \pm 1,123.29$ & $409,901 \pm 39,475.80$ &$0.0888 \pm 0.0005$ 
0.25 $131,122 \pm 280.82$ & $103,076 \pm 9,911.05$ &$0.1776 \pm 0.0010$ 
0.5 $32,780 \pm 70.21$ & $25,988 \pm 2,497.34$ &$0.3559 \pm 0.0020$ 
1.0 $8,195 \pm 17.55$ & $6,561 \pm 630.28$ &$0.7170 \pm 0.0038$ 


In [9]:
eps = 0.25
data = [compare_ea_eap(f"../experiments/LSVM/{e}/worlds_results/world{i}/eps_{eps}/", i, eps) 
                               for i in range(0, 11) for e in [0, 1]]
print(construct_ea_eap_ptable(data, 4))

+-------+---+-------+---------+-------------+----------------+--------------+--------------+-----------+
| world | c | delta | epsilon | eap_epsilon | size_of_market |           ea |          eap |   ratio   |
+-------+---+-------+---------+-------------+----------------+--------------+--------------+-----------+
|   7   | 2 |  0.1  |  0.2500 |    0.1697   |     263,168    |  130,331,574 |   49,451,598 |   37.9429 |
|   0   | 2 |  0.1  |  0.2500 |    0.1754   |     264,576    |  131,074,050 |   77,724,150 |   59.2979 |
|   4   | 2 |  0.1  |  0.2500 |    0.1761   |     263,616    |  130,567,790 |   82,976,042 |   63.5502 |
|   5   | 2 |  0.1  |  0.2500 |    0.1763   |     265,536    |  131,580,421 |   85,542,216 |   65.0114 |
|   3   | 2 |  0.1  |  0.2500 |    0.1763   |     263,616    |  130,567,790 |   84,453,510 |   64.6817 |
|   0   | 2 |  0.1  |  0.2500 |    0.1765   |     264,832    |  131,209,071 |   87,253,743 |   66.4998 |
|   10  | 2 |  0.1  |  0.2500 |    0.1772   |     264,1

## A few samples of EA vs EAP on GSVM with schedule given by eps = [10.0, 5.0, 2.5, 1.25]

In [10]:
print(construct_ea_eap_ptable([compare_ea_eap(f"../value_models/GSVM/develop/4/worlds_results/world0/eps_10.0/", 0, 10.0),
                              compare_ea_eap(f"../value_models/GSVM/develop/0/worlds_results/world0/eps_10.0/", 0, 10.0)]))

+-------+-----+-------+----------+-------------+----------------+-------------+-------------+-----------+
| world |  c  | delta | epsilon  | eap_epsilon | size_of_market |          ea |         eap |   ratio   |
+-------+-----+-------+----------+-------------+----------------+-------------+-------------+-----------+
|   0   | 402 |  0.1  |  10.0000 |    7.2405   |      4,480     |  41,278,458 |  57,007,145 |  138.1039 |
|   0   | 402 |  0.1  |  10.0000 |    7.3538   |      4,480     |  41,278,458 |  67,349,860 |  163.1598 |
+-------+-----+-------+----------+-------------+----------------+-------------+-------------+-----------+


In [11]:
print(construct_ea_eap_ptable([compare_ea_eap(f"../value_models/GSVM/develop/2/worlds_results/world0/eps_5.0/", 0, 5.0),
                              compare_ea_eap(f"../value_models/GSVM/develop/0/worlds_results/world0/eps_5.0/", 0, 5.0)]))

+-------+-----+-------+---------+-------------+----------------+--------------+--------------+----------+
| world |  c  | delta | epsilon | eap_epsilon | size_of_market |           ea |          eap |  ratio   |
+-------+-----+-------+---------+-------------+----------------+--------------+--------------+----------+
|   0   | 402 |  0.1  |  5.0000 |    3.3294   |      4,480     |  165,113,830 |   84,616,896 |  51.2476 |
|   0   | 402 |  0.1  |  5.0000 |    3.4148   |      4,480     |  165,113,830 |  119,519,528 |  72.3861 |
+-------+-----+-------+---------+-------------+----------------+--------------+--------------+----------+


In [12]:
print(construct_ea_eap_ptable([compare_ea_eap(f"../value_models/GSVM/develop/1/worlds_results/world0/eps_2.5/", 0, 2.5),
                              compare_ea_eap(f"../value_models/GSVM/develop/0/worlds_results/world0/eps_2.5/", 0, 2.5)]))

+-------+-----+-------+---------+-------------+----------------+--------------+--------------+----------+
| world |  c  | delta | epsilon | eap_epsilon | size_of_market |           ea |          eap |  ratio   |
+-------+-----+-------+---------+-------------+----------------+--------------+--------------+----------+
|   0   | 402 |  0.1  |  2.5000 |    1.4917   |      4,480     |  660,455,321 |  193,784,326 |  29.3410 |
|   0   | 402 |  0.1  |  2.5000 |    1.5658   |      4,480     |  660,455,321 |  205,430,768 |  31.1044 |
+-------+-----+-------+---------+-------------+----------------+--------------+--------------+----------+


In [13]:
print(construct_ea_eap_ptable([compare_ea_eap(f"../value_models/GSVM/develop/3/worlds_results/world0/eps_1.25/", 0, 1.25),
                              compare_ea_eap(f"../value_models/GSVM/develop/0/worlds_results/world0/eps_1.25/", 0, 1.25)]))

+-------+-----+-------+---------+-------------+----------------+----------------+--------------+----------+
| world |  c  | delta | epsilon | eap_epsilon | size_of_market |             ea |          eap |  ratio   |
+-------+-----+-------+---------+-------------+----------------+----------------+--------------+----------+
|   0   | 402 |  0.1  |  1.2500 |    0.7059   |      4,480     |  2,641,821,286 |  687,875,718 |  26.0379 |
|   0   | 402 |  0.1  |  1.2500 |    0.7286   |      4,480     |  2,641,821,286 |  721,783,008 |  27.3214 |
+-------+-----+-------+---------+-------------+----------------+----------------+--------------+----------+
